In [1]:
import pickle
import pandas as pd
import nltk
nltk.download('stopwords')
nltk.download('punkt')
import itertools
from collections import Counter
import numpy as np
from nltk import word_tokenize
from nltk.corpus import stopwords
from gensim.models import word2vec
from sklearn.linear_model import LogisticRegression
import os
import string

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Data Exploration

In [2]:
df_train = pd.read_csv("train.csv")
df_train.head()

,id,attributes.HappyHour,attributes.Ambience,hours.Tuesday,postal_code,attributes.AgesAllowed,attributes.GoodForDancing,attributes.OutdoorSeating,hours.Saturday,attributes.Corkage,...,attributes.AcceptsInsurance,attributes.RestaurantsDelivery,attributes.DietaryRestrictions,attributes.BusinessAcceptsBitcoin,address,attributes.GoodForKids,attributes.GoodForMeal,hours,label,review
0,0,b'True',"b""{'romantic': False, 'intimate': False, 'clas...",b'15:0-2:0',b'44107',NaN,NaN,b'False',b'11:30-2:0',NaN,...,NaN,b'False',NaN,NaN,b'17800 Detroit Ave',b'False',"b""{'dessert': False, 'latenight': False, 'lunc...","{'Monday': '16:0-2:0', 'Tuesday': '15:0-2:0', ...",american (traditional),"So, we stopped here on our way to the Side Que..."
1,1,NaN,"b""{'romantic': False, 'intimate': False, 'tour...",b'11:0-21:0',b'85042',NaN,NaN,b'True',b'11:0-20:30',NaN,...,NaN,b'False',NaN,b'False',"b'2160 E Baseline Rd, Ste 128'",b'True',"b""{'dessert': False, 'latenight': False, 'lunc...","{'Monday': '11:0-21:0', 'Tuesday': '11:0-21:0'...",american (new),This is our go-to healthy spot! The food is al...
2,2,NaN,NaN,b'11:0-21:0',b'M4M 3G6',NaN,NaN,NaN,b'11:0-21:0',NaN,...,NaN,NaN,NaN,NaN,b'1000 Gerrard St E',NaN,NaN,"{'Monday': '11:0-21:0', 'Tuesday': '11:0-21:0'...",mexican,Food court meal at Gerrard Square. It's been ...
3,3,NaN,"b""{'romantic': False, 'intimate': False, 'clas...",NaN,b'89146',NaN,NaN,b'False',NaN,NaN,...,NaN,b'True',NaN,NaN,b'6700 W Charleston Blvd',b'True',NaN,NaN,mexican,"Located on Rainbow/Charleston, this small fami..."
4,4,NaN,"b""{'romantic': False, 'intimate': False, 'tour...",NaN,b'44133',NaN,NaN,b'False',NaN,NaN,...,NaN,b'False',NaN,NaN,b'5630 Wallings Rd',b'True',"b""{'dessert': False, 'latenight': False, 'lunc...",NaN,chinese,No frills Chinese takeout joint which serves u...


In [3]:
df_train.shape

(13144, 62)

In [4]:
df_train.describe()

,id,longitude,attributes.HairSpecializesIn,review_count,is_open,stars,latitude
count,13144.000000,13144.000000,0.0,13144.000000,13144.000000,13144.000000,13144.000000
mean,6571.500000,-93.924099,NaN,88.824026,0.716525,3.468883,39.378315
std,3794.490304,16.536313,NaN,184.876083,0.450702,0.710503,4.845917
min,0.000000,-115.351527,NaN,5.000000,0.000000,1.000000,33.210781
25%,3285.750000,-112.082824,NaN,13.000000,0.000000,3.000000,35.117573
50%,6571.500000,-81.699423,NaN,34.000000,1.000000,3.500000,40.428478
75%,9857.250000,-79.559842,NaN,90.000000,1.000000,4.000000,43.656322
max,13143.000000,-72.949770,NaN,5763.000000,1.000000,5.000000,51.294184


In [5]:
df_train.label.value_counts()

label
american (traditional)    2680
mexican                   2217
italian                   2032
chinese                   1696
american (new)            1399
japanese                  1063
mediterranean              728
canadian (new)             484
thai                       483
asian fusion               362
Name: count, dtype: int64

In [6]:
df_train.columns

Index(['id', 'attributes.HappyHour', 'attributes.Ambience', 'hours.Tuesday',
       'postal_code', 'attributes.AgesAllowed', 'attributes.GoodForDancing',
       'attributes.OutdoorSeating', 'hours.Saturday', 'attributes.Corkage',
       'longitude', 'name', 'attributes.BusinessAcceptsCreditCards',
       'attributes.RestaurantsTableService',
       'attributes.RestaurantsReservations', 'hours.Friday',
       'attributes.RestaurantsPriceRange2', 'attributes.WiFi',
       'attributes.ByAppointmentOnly', 'attributes.Music',
       'attributes.NoiseLevel', 'attributes.BYOB', 'state',
       'attributes.Alcohol', 'attributes.HasTV', 'attributes',
       'attributes.BYOBCorkage', 'hours.Wednesday', 'hours.Sunday',
       'attributes.RestaurantsGoodForGroups', 'attributes.Open24Hours',
       'attributes.BusinessParking', 'attributes.DogsAllowed',
       'attributes.HairSpecializesIn', 'review_count', 'is_open',
       'attributes.Caters', 'attributes.CoatCheck', 'attributes.BikeParking',
   

## Data Preprocessing and Feature Engineering

### "review" variable - unstructured data, text mining

In [7]:
df_train[["label","review"]].head()

,label,review
0,american (traditional),"So, we stopped here on our way to the Side Que..."
1,american (new),This is our go-to healthy spot! The food is al...
2,mexican,Food court meal at Gerrard Square. It's been ...
3,mexican,"Located on Rainbow/Charleston, this small fami..."
4,chinese,No frills Chinese takeout joint which serves u...


In [8]:
df_train["review"].iloc[0]

'So, we stopped here on our way to the Side Quest, which is just up the street. \\n\\nWe did not know what to expect, but came on a 3$ taco night...\\n\\nI\'m so glad we did:)\\n\\nMostly, the Rush Inn is a bar-ish place, but there\'s plenty of back and side seating at booths all through. The place has a dark, rich wood tone, and the patrons were decent folks.\\n\\nThe tacos were EXCEPTIONAL, and their selection is quite good, with all sorts of stuff I intend to try as soon as possible. I think this place would make a nice casual dinner/lunch stop, and if you need to get beverage, the barkeep and wait staff will keep em coming! \\n\\nSo, for a quick tasty bite, don\'t walk slow, RUSH IN TO THE RUSH INN! My boyfriend and I heard about this little place near Rush Inn that offered board games and poutine on Friday. When we arrived, they didn\'t have food because the chef didn\'t show up (?) so we ventured to this place a block down. Great vibe inside. A typical bar scene, but had great bo

- Preprocessing - stemming, tokenization, stop words removal

In [9]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()

# Define the tokenizer function
def pre_processing(text):
    # English stopwords
    stop_words = set(stopwords.words('english'))
    stop_words.add('would')

    # Prepare translation table to remove punctuation
    translator = str.maketrans(string.punctuation, ' ' * len(string.punctuation))

    # Remove punctuation and split into words
    text_nopunct = text.translate(translator)
    words = text_nopunct.strip().split()

    # Filter stopwords and single characters, and stem
    filtered_words = [ps.stem(word) for word in words if word.lower() not in stop_words and len(word) > 1]

    return filtered_words

- TF-IDF

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(strip_accents=None,
                        lowercase=True,
                        preprocessor=None,  # applied preprocessor in Data Cleaning
                        tokenizer=pre_processing,   # can be the one we create ourselves
                        use_idf=True,
                        norm='l2',
                        smooth_idf=True)

X_review = tfidf.fit_transform(df_train["review"])
y = df_train["label"]

/opt/conda/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


- Add more weights to important_words

In [11]:
important_words = ["american", "mexican", "chines", "mediterranean", "italian", "thai", "asian", "canadian", "japanes"]

In [12]:
important_word_indices = [tfidf.vocabulary_.get(word) for word in important_words if word in tfidf.vocabulary_]

In [13]:
important_word_indices

[2445, 23375, 7861, 23092, 19155, 40665, 3224, 6653, 19354]

In [16]:
# Scale up the TF-IDF scores for these words
for idx in important_word_indices:
    X_review[:, idx] *= 2  # weight_factor > 1 to increase the weight

In [17]:
X_review.shape

(13144, 46367)

### "name" variable - unstructured data, text mining

In [18]:
# add one more feature
df_train["name"].head()

0                b'Rush Inn'
1                b'GreenMix'
2    b'BarBurrito - Gerrard'
3                b'SalvaMex'
4                b'Hop Hing'
Name: name, dtype: object

- extract the name pattern

In [19]:
import re

# Define the regex pattern
pattern = re.compile(r"b'([^']+)")

# Update the column by applying the regex pattern and extracting the matched group
df_train['name'] = df_train['name'].apply(lambda x: pattern.search(str(x)).group(1) if pattern.search(str(x)) else x)


In [20]:
df_train["name"].head()

0                Rush Inn
1                GreenMix
2    BarBurrito - Gerrard
3                SalvaMex
4                Hop Hing
Name: name, dtype: object

In [21]:
X_name = tfidf.transform(df_train["name"])

### "attributes.Ambience" vairable

In [22]:
df_train["attributes.Ambience"][0]

'b"{\'romantic\': False, \'intimate\': False, \'classy\': False, \'hipster\': False, \'divey\': False, \'touristy\': False, \'trendy\': False, \'upscale\': False, \'casual\': True}"'

- extract the pattern

In [23]:
# Define a regex pattern to extract key and boolean value
pattern = re.compile(r"'(\w+)': (True|False)")

keys = set()

# generate a key set that have all the categories in the Ambience column
for row in df_train["attributes.Ambience"]:
    if isinstance(row, str):  # Check if the row is a string
        matches = pattern.findall(row)
        for key, value in matches:
            keys.add(key)

print(keys)

{'upscale', 'intimate', 'casual', 'romantic', 'classy', 'touristy', 'hipster', 'trendy', 'divey'}


In [24]:
all_keys = dict()
for k in keys:
    all_keys[k] = len(all_keys)

In [25]:
# seems the sequency of the keys in the Ambience is not the same and sometimes has absence
# so try to use the same sequence using the same as the keys set

# Initialize an empty list to store the numeric representations
numeric_data = []

for row in df_train["attributes.Ambience"]:
    numeric_row = [0] * 9
    if isinstance(row, str):  # Check if the row is a string
        # Find all matches in the current row
        matches = pattern.findall(row)
        # Convert matches to numeric format (0s and 1s)
        for key, value in matches:
            if value == "True":
                token_id = all_keys[key]
                numeric_row[token_id] = 1
    else:
        numeric_row = [0] * 9
    numeric_data.append(numeric_row)

In [26]:
# check
for i in numeric_data:
    if len(i) == 9:
        continue
    else:
        print(i)

In [27]:
# Convert numeric_data to a numpy array
numeric_array = np.array(numeric_data)

In [28]:
numeric_array.shape

(13144, 9)

In [29]:
from scipy.sparse import csr_matrix, hstack

X_Ambience = csr_matrix(numeric_array)

### "state" vairable

In [30]:
df_train["state"].value_counts()

state
b'AZ'    2986
b'ON'    2959
b'NV'    1966
b'OH'    1300
b'NC'    1046
b'PA'     906
b'QC'     798
b'AB'     485
b'WI'     425
b'IL'     178
b'SC'      94
b'VA'       1
Name: count, dtype: int64

In [31]:
# common states in the training set
all_states = ["b'AZ'", "b'ON'", "b'NV'", "b'OH'", "b'NC'", "b'PA'", "b'QC'", "b'AB'", "b'WI'", "b'IL'", "b'SC'"]
len(all_states)

11

- one-hot encoding

In [32]:
state_data = []
for row in df_train["state"]:
    state = [0] * (len(all_states) + 1)
    if isinstance(row, str):  # Check if the row is a string
        if row in all_states:
            # Find the index of the state and set the corresponding position to 1
            index = all_states.index(row)
            state[index] = 1
        else:
            # If the state is not in all_states, categorize it as "other"
            state[-1] = 1  # Set the last position to 1 for "other"
    else:   # nan
        state[-1] = 1
    state_data.append(state)

In [33]:
X_state = np.array(state_data)

In [34]:
X_state.shape

(13144, 12)

In [35]:
X_state = csr_matrix(X_state)

### "OutdoorSeating" variable

In [36]:
df_train["attributes.OutdoorSeating"].value_counts(dropna=False)

attributes.OutdoorSeating
b'False'    7044
b'True'     4791
NaN         1299
b'None'       10
Name: count, dtype: int64

- one-hot encoding

In [37]:
seat_data = []
for row in df_train["attributes.OutdoorSeating"]:
    seat = [0] * 3
    if isinstance(row, str):  # Check if the row is a string
        if row == "b'False'":
            seat[0] = 1
        elif row == "b'True'":
            seat[1] = 1
        else:
            seat[-1] = 1  # Set the last position to 1 for "other"
    else:   # nan
        seat[-1] = 1
    seat_data.append(seat)

In [39]:
X_seat = np.array(seat_data)

In [40]:
X_seat.shape

(13144, 3)

In [41]:
X_seat = csr_matrix(X_seat)

### "Caters" variable

In [42]:
df_train["attributes.Caters"].value_counts(dropna=False)

attributes.Caters
b'True'     4799
NaN         4330
b'False'    3996
b'None'       19
Name: count, dtype: int64

- one-hot encoding

In [43]:
Caters_data = []
for row in df_train["attributes.Caters"]:
    Caters = [0] * 3
    if isinstance(row, str):  # Check if the row is a string
        if row == "b'False'":
            Caters[0] = 1
        elif row == "b'True'":
            Caters[1] = 1
        else:
            Caters[-1] = 1  # Set the last position to 1 for "other"
    else:   # nan
        Caters[-1] = 1
    Caters_data.append(Caters)

In [44]:
X_Caters = np.array(Caters_data)
X_Caters = csr_matrix(X_Caters)

### "WiFi" variable

In [45]:
df_train["attributes.WiFi"].value_counts(dropna=False)

attributes.WiFi
b"u'no'"      3550
b"u'free'"    3034
NaN           2922
b"'no'"       1981
b"'free'"     1568
b"u'paid'"      46
b"'paid'"       25
b'None'         18
Name: count, dtype: int64

- extract the pattern

In [46]:
# Define the regex pattern
pattern = re.compile(r"b[\"']u?'?([^']+)'?\"?") 

# Update the column by applying the regex pattern and extracting the matched group
df_train['attributes.WiFi'] = df_train['attributes.WiFi'].apply(lambda x: pattern.search(str(x)).group(1) if pattern.search(str(x)) else x)


- one-hot encoding

In [47]:
WiFi_data = []
for row in df_train["attributes.WiFi"]:
    WiFi = [0] * 4
    if isinstance(row, str):  # Check if the row is a string
        if row == "paid":
            WiFi[0] = 1
        elif row == "free":
            WiFi[1] = 1
        elif row == "no":
            WiFi[2] = 1
        else:
            WiFi[-1] = 1  # Set the last position to 1 for "other"
    else:   # nan
        WiFi[-1] = 1
    WiFi_data.append(WiFi)

In [48]:
X_WiFi = np.array(WiFi_data)
X_WiFi = csr_matrix(X_WiFi)

- combine all the features we preprocessed and engineered

In [49]:
X_all = hstack([X_review, X_name, X_Ambience, X_state, X_seat, X_Caters, X_WiFi])

# Ensure the result is in CSR format for efficiency
X_all = X_all.tocsr()

In [50]:
X_all.shape

(13144, 92765)

## Model Building

In [51]:
from sklearn.model_selection import train_test_split
import pickle
from sklearn.linear_model import LogisticRegressionCV


X_train, X_test, y_train, y_test = train_test_split(X_all, y, random_state=42, test_size=0.3, shuffle=True)

In [52]:
clf = LogisticRegression(max_iter=100000000).fit(X_train, y_train)


In [53]:
clf.score(X_test, y_test)

0.8268255578093306

## Predict on test data

In [54]:
df_test = pd.read_csv("test.csv")

In [55]:
clf = LogisticRegression(max_iter=100000000).fit(X_all, y)

In [56]:
X_review_test = tfidf.transform(df_test["review"])

In [57]:
# Scale up the TF-IDF scores for these words
for idx in important_word_indices:
    X_review_test[:, idx] *= 2  # weight_factor > 1 to increase the weight

In [58]:
import re

# Define the regex pattern
pattern = re.compile(r"b'([^']+)")

# Update the column by applying the regex pattern and extracting the matched group
df_test['name'] = df_test['name'].apply(lambda x: pattern.search(str(x)).group(1) if pattern.search(str(x)) else x)

In [59]:
X_name_test = tfidf.transform(df_test["name"])

In [60]:
# seems the sequency of the keys in the Ambience is not the same and sometimes has absence
# so try to use the same sequence using the same as the keys set

pattern = re.compile(r"'(\w+)': (True|False)")

# Initialize an empty list to store the numeric representations
numeric_data = []

for row in df_test["attributes.Ambience"]:
    numeric_row = [0] * 9
    if isinstance(row, str):  # Check if the row is a string
        # Find all matches in the current row
        matches = pattern.findall(row)
        # Convert matches to numeric format (0s and 1s)
        for key, value in matches:
            if value == "True":
                token_id = all_keys[key]
                numeric_row[token_id] = 1
    else:
        numeric_row = [0] * 9
    numeric_data.append(numeric_row)

In [61]:
# check
for i in numeric_data:
    if len(i) == 9:
        continue
    else:
        print(i)

In [62]:
# Convert numeric_data to a numpy array
numeric_array = np.array(numeric_data)

In [63]:
X_Ambience_test = csr_matrix(numeric_array)

In [64]:
state_data = []
for row in df_test["state"]:
    state = [0] * (len(all_states) + 1)
    if isinstance(row, str):  # Check if the row is a string
        if row in all_states:
            # Find the index of the state and set the corresponding position to 1
            index = all_states.index(row)
            state[index] = 1
        else:
            # If the state is not in all_states, categorize it as "other"
            state[-1] = 1  # Set the last position to 1 for "other"
    else:   # nan
        state[-1] = 1
    state_data.append(state)

In [65]:
X_state_test = np.array(state_data)

In [66]:
X_state_test.shape

(10000, 12)

In [67]:
X_state_test = csr_matrix(X_state_test)

In [68]:
seat_data = []
for row in df_test["attributes.OutdoorSeating"]:
    seat = [0] * 3
    if isinstance(row, str):  # Check if the row is a string
        if row == "b'False'":
            seat[0] = 1
        elif row == "b'True'":
            seat[1] = 1
        else:
            seat[-1] = 1  # Set the last position to 1 for "other"
    else:   # nan
        seat[-1] = 1
    seat_data.append(seat)

In [69]:
X_seat_test = np.array(seat_data)
X_seat_test = csr_matrix(X_seat_test)

In [70]:
Caters_data = []
for row in df_test["attributes.Caters"]:
    Caters = [0] * 3
    if isinstance(row, str):  # Check if the row is a string
        if row == "b'False'":
            Caters[0] = 1
        elif row == "b'True'":
            Caters[1] = 1
        else:
            Caters[-1] = 1  # Set the last position to 1 for "other"
    else:   # nan
        Caters[-1] = 1
    Caters_data.append(Caters)

In [71]:
X_Caters_test = np.array(Caters_data)
X_Caters_test = csr_matrix(X_Caters_test)

In [72]:
# Define the regex pattern
pattern = re.compile(r"b[\"']u?'?([^']+)'?\"?") 

# Update the column by applying the regex pattern and extracting the matched group
df_test['attributes.WiFi'] = df_test['attributes.WiFi'].apply(lambda x: pattern.search(str(x)).group(1) if pattern.search(str(x)) else x)


In [73]:
WiFi_data = []
for row in df_test["attributes.WiFi"]:
    WiFi = [0] * 4
    if isinstance(row, str):  # Check if the row is a string
        if row == "paid":
            WiFi[0] = 1
        elif row == "free":
            WiFi[1] = 1
        elif row == "no":
            WiFi[2] = 1
        else:
            WiFi[-1] = 1  # Set the last position to 1 for "other"
    else:   # nan
        WiFi[-1] = 1
    WiFi_data.append(WiFi)

In [74]:
X_WiFi_test = np.array(WiFi_data)
X_WiFi_test = csr_matrix(X_WiFi_test)

In [75]:
from scipy.sparse import csr_matrix, hstack

X_all_test = hstack([X_review_test, X_name_test, X_Ambience_test, X_state_test, X_seat_test, X_Caters_test, X_WiFi_test])

# Ensure the result is in CSR format for efficiency
X_all_test = X_all_test.tocsr()

In [76]:
X_all_test.shape

(10000, 92765)

In [77]:
preds = clf.predict(X_all_test)

In [78]:
dic = {"Id": [], "Predicted": []}
for i, pred in enumerate(preds):
    dic["Id"].append(i)
    dic["Predicted"].append(pred)


In [79]:
dic_df = pd.DataFrame.from_dict(dic)
dic_df.to_csv("predicted.csv", index=False)

Result: The final test data prediction score was 0.82684, ranking 2nd on the kaggle rankings.